In [3]:
import sys
import json
import pandas as pd
from pathlib import Path


sys.path.append(str(Path().resolve().parents[1]))

PROJECT_ROOT = Path().resolve().parents[1]
RESULTS_PATH = PROJECT_ROOT / "results"

MODEL_PATH = RESULTS_PATH / "single_point_model_metrics_log.json"

with open(MODEL_PATH, "r") as f:
    model_logs = json.load(f)

# Filter out runs 2 and 3
filtered_models = [m for m in model_logs if m["run"] not in [2, 3]]

# Helper function to extract a row for a specific dataset (test or validation)
def extract_metrics_row(model, dataset):
    metrics = model["metrics"][dataset]
    return {
        "Model": model["id"],
        "MAE": round(metrics["MAE"], 3),
        "MSE": round(metrics["MSE"], 3),
        "RMSE": round(metrics["RMSE"], 3),
        "Mean Bias Error": round(metrics["Mean Bias Error"], 3),
        "MAPE (%)": round(metrics["MAPE"], 3),
        "nRMSE (%)": round(metrics["nRMSE"], 3),
    }

# Build test and validation tables
test_rows = [extract_metrics_row(m, "test") for m in filtered_models]
val_rows = [extract_metrics_row(m, "validation") for m in filtered_models]

# Convert to DataFrames
test_df = pd.DataFrame(test_rows)
val_df = pd.DataFrame(val_rows)

# Export to CSV
test_df.to_csv(RESULTS_PATH / "model_comparison_test.csv", index=False)
val_df.to_csv(RESULTS_PATH / "model_comparison_validation.csv", index=False)

# Optional: preview
print("Test Set Performance:\n", test_df)
print("\nValidation Set Performance:\n", val_df)


Test Set Performance:
                          Model    MAE    MSE   RMSE  Mean Bias Error  \
0   Linear Regression Baseline  1.049  2.769  1.256            0.010   
1  Optimized Linear Regression  0.626  1.357  0.722            0.141   
2   Baseline XGBoost Regressor  0.922  2.331  1.122            0.039   
3  XBoost Regressor Configured  0.636  1.290  0.728            0.103   
4   Final Bayesian Ridge Model  0.647  1.517  0.748            0.146   

   MAPE (%)  nRMSE (%)  
0     9.853      6.678  
1     5.916      4.214  
2     8.718      5.967  
3     6.034      4.246  
4     6.119      4.364  

Validation Set Performance:
                          Model    MAE    MSE   RMSE  Mean Bias Error  \
0   Linear Regression Baseline  1.225  3.936  1.528            0.277   
1  Optimized Linear Regression  0.490  0.623  0.534            0.009   
2   Baseline XGBoost Regressor  0.978  2.656  1.189           -0.058   
3  XBoost Regressor Configured  0.560  0.838  0.594           -0.074   
4   